In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_excel('data-5features-all-outputs.xlsx')
bigfive = ["Extraversion", "Agreeableness", "Conscientiousness", "Neuroticism", "Openness"]
target = "Team_Climate"
df = df.iloc[:,2:]
df

,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Openness,Vision,Task_Orientation,Support_for_innovation,Participative_safety,Team_Climate
0,57,96,29,73,80,4.000000,5.000000,4.750,4.833333,4.605
1,3,75,26,84,36,4.000000,4.428571,4.000,4.583333,4.263
2,72,94,98,10,40,4.818182,5.000000,5.000,5.000000,4.947
3,70,68,79,53,58,4.818182,4.857143,3.875,5.000000,4.684
4,80,18,18,56,13,4.000000,4.142857,3.750,3.833333,3.921
...,...,...,...,...,...,...,...,...,...,...
277,29,60,79,50,42,4.818182,5.000000,5.000,4.833333,4.830
278,8,54,48,91,37,3.636364,3.428571,4.250,3.750000,3.770
279,72,28,65,23,9,4.727273,3.285714,3.125,3.750000,3.720
280,57,65,58,56,5,4.363636,4.285714,4.125,4.000000,4.260


Previous research have only implemented regression analysis with neuroticism and each output feature of team climate. They have selected neuroticism because was the only personality variable that demonstrated high correlation between neuroticism and team climate variables

In [40]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(df[['Neuroticism', 'Support_for_innovation']], df[target], test_size=0.2)
clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
test_score = clf.score(X_test, y_test)
train_score = clf.score(X_train, y_train)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print(f"Test score: {test_score}")
print(f"Train score: {train_score}")

Mean Squared Error: 0.09158933655345987
R^2 Score: 0.7925288038456164
Test score: 0.7925288038456164
Train score: 0.7713680491141732
